多输入多输出通道  
彩色图像具有标准的RGB通道来代表红、绿和蓝  
当我们添加通道时，我们的输入和隐藏的表示都变成了三维张量。  
例如，每个RGB输入图像具有的形状$3 \times h \times w$。  
我们将这个大小为3的轴称为通道（channel）维度

多输入通道  
对每个通道执行互相关操作，然后将结果相加  
卷积核张量形状为$c_i \times k_h \times k_w$

In [14]:
import torch
from torch import nn
from d2l import torch as d2l
#K有三个维度
def corr2d_multi_in(X, K):
    # 先遍历“X”和“K”的第0个维度（通道维度），再把它们加在一起
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))


In [15]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

X.shape,K.shape,corr2d_multi_in(X, K)


(torch.Size([2, 3, 3]),
 torch.Size([2, 2, 2]),
 tensor([[ 56.,  72.],
         [104., 120.]]))

多输出通道  
随着神经网络层数的加深，我们常会增加输出通道的维数，通过减少空间分辨率以获得更大的通道深度  
可以将每个通道看作对不同特征的响应。  
而现实可能更为复杂一些，因为每个通道不是独立学习的，而是为了共同使用而优化的。  
因此，多输出通道并不仅是学习多个单通道的检测器。

卷积核大小$k_h \times k_w$  
为了获得多个通道的输出，我们可以为每个输出通道创建一个形状为$c_i \times k_h \times k_w$的卷积核张量  
这样卷积核形状为$c_o \times c_i \times k_h \times k_w$

In [16]:
def corr2d_multi_in_out(X, K):
    # 迭代“K”的第0个维度，每次都对输入“X”执行互相关运算。
    # 最后将所有结果都叠加在一起
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)


通过将核张量K与K+1（K中每个元素加）和K+2连接起来，构造了一个具有3个输出通道的卷积核

In [17]:
K = torch.stack((K, K + 1, K + 2), 0)
K.shape


torch.Size([3, 2, 2, 2])

对输入张量X与卷积核张量K执行互相关运算。  
现在的输出包含个3通道，第一个通道的结果与先前输入张量X和多输入单输出通道的结果一致

In [18]:
X.shape,K.shape,corr2d_multi_in_out(X, K)


(torch.Size([2, 3, 3]),
 torch.Size([3, 2, 2, 2]),
 tensor([[[ 56.,  72.],
          [104., 120.]],
 
         [[ 76., 100.],
          [148., 172.]],
 
         [[ 96., 128.],
          [192., 224.]]]))

$1 \times 1$卷积核  
这种卷积失去了卷积层的特有能力——在高度和宽度维度上，识别相邻元素间相互作用的能力  
唯一计算发生在通道上  
看作在每个像素位置应用的全连接层，以$c_i$个输入值转换为$c_o$个输出值。   
因为这仍然是一个卷积层，所以跨像素的权重是一致的  
需要的权重维度为$c_i \times c_o$再额外加上一个偏置

使用全连接层实现1*1卷积。  
请注意，我们需要对输入和输出的数据形状进行调整

In [19]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    # 全连接层中的矩阵乘法
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))


上述函数相当于先前实现的互相关函数corr2d_multi_in_out。  
让我们用一些样本数据来验证这一点。

In [20]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6


汇聚层  
希望逐渐降低隐藏表示的空间分辨率、聚集信息，这样随着我们在神经网络中层叠的上升，每个神经元对其敏感的感受野（输入）就越大。  
而我们的机器学习任务通常会跟全局图像的问题有关（例如，“图像是否包含一只猫呢？”），  
所以我们最后一层的神经元应该对整个输入的全局敏感。  
通过逐渐聚合信息，生成越来越粗糙的映射，最终实现学习全局表示的目标，同时将卷积图层的所有优势保留在中间层  
汇聚（pooling）层，它具有双重目的：  
降低卷积层对位置的敏感性，同时降低对空间降采样表示的敏感性。

与卷积层类似，汇聚层运算符由一个固定形状的窗口组成，该窗口根据其步幅大小在输入的所有区域上滑动，为固定形状窗口（有时称为汇聚窗口）遍历的每个位置计算一个输出。   
然而，不同于卷积层中的输入与卷积核之间的互相关计算，汇聚层不包含参数。   
相反，池运算是确定性的，我们通常计算汇聚窗口中所有元素的最大值或平均值。  
这些操作分别称为最大汇聚层（maximum pooling）和平均汇聚层（average pooling）。

汇聚窗口从输入张量的左上角开始，从左往右、从上往下的在输入张量内滑动  
汇聚窗口到达的每个位置，它计算该窗口中输入子张量的最大值或平均值。  
计算最大值或平均值是取决于使用了最大汇聚层还是平均汇聚层

In [21]:
def pool2d(X, pool_size, mode='max'):
    p_h, p_w = pool_size
    Y = torch.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1))#输出形状
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i, j] = X[i: i + p_h, j: j + p_w].max()
            elif mode == 'avg':
                Y[i, j] = X[i: i + p_h, j: j + p_w].mean()
    return Y

输入：
$\begin{bmatrix}0& 1 & 2 \\ 3 & 4 &5 \\6 & 7 & 8\end{bmatrix}$  
2*2 max pooling输出:  
$\begin{bmatrix} 4 & 5 \\ 7 & 8 \end{bmatrix}$

In [22]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
pool2d(X, (2, 2))


tensor([[4., 5.],
        [7., 8.]])

In [23]:
pool2d(X, (2, 2), 'avg')

tensor([[2., 3.],
        [5., 6.]])

填充和步幅  
与卷积层一样，汇聚层也可以改变输出形状。  
和以前一样，我们可以通过填充和步幅以获得所需的输出形状  
输入矩阵形状$n_h \times n_w$  
pooling形状$pl_h \times pl_w$  
stride $s_h \times s_w$  
pading $p_h \times p_w$  
输出形状：  
$[(n_h-(pl_h-1)+2p_h+s_h-1)/s_h]\times[(n_w-(pl_w-1)+2p_w+s_w-1)/s_w]$   
当pading=0,输出为：  
$[(n_h-(pl_h-1)+s_h-1)/s_h]\times[(n_w-(pl_w-1)+s_w-1)/s_w]$  
chanel保持不变  

In [24]:
X = torch.arange(16, dtype=torch.float32).reshape((1, 1, 4, 4))
X


tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

默认情况下，深度学习框架中的步幅与汇聚窗口的大小相同。   
因此，如果我们使用形状为(3, 3)的汇聚窗口，那么默认情况下，我们得到的步幅形状为(3, 3)

In [25]:
pool2d = nn.MaxPool2d(3)
pool2d(X)


tensor([[[[10.]]]])

In [26]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)


tensor([[[[ 5.,  7.],
          [13., 15.]]]])

多个通道  
在处理多通道输入数据时，汇聚层在每个输入通道上单独运算，而不是像卷积层一样在通道上对输入进行汇总  
意味着汇聚层的输出通道数与输入通道数相同

In [27]:
X = torch.cat((X, X + 1), 1)
X


tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])

In [28]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)


tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])